In [ ]:
!pip install keras

In [ ]:
%pylab inline

In [ ]:
import seaborn
import pandas as pd
import numpy as np
from keras import Sequential
from keras.layers import Dense, LSTM
from keras.preprocessing.sequence import TimeseriesGenerator
np.random.seed(12345)
seaborn.set_context("talk")
seaborn.set_style('white')

# Generate Simple Example Data

In [ ]:
x = np.linspace(-3, 3, num=2000)[:, np.newaxis]
y = 1.5*x[:, 0]**3 - 5*x[:, 0]**2 + 3*x[:, 0] + 3*np.random.randn(x.shape[0])
train_x = x[:1700]
train_y = y[:1700]
test_x = x[1700:]
test_y = y[1700:]


In [ ]:
plot(train_x, train_y)
plot(test_x, test_y)

# Fit Simple Neural Network

In [ ]:

model = Sequential()
model.add(Dense(20, input_shape=(1,)))
model.add(Dense(30, activation='relu'))
model.add(Dense(30, activation='relu'))
model.add(Dense(30, activation='relu'))
model.add(Dense(1))
model.compile(loss='mse', optimizer='adam')

history = model.fit(train_x, train_y, epochs=5, validation_split=0.4, batch_size=10, verbose=0)



In [ ]:
plot(history.history['val_loss'])

In [ ]:
train_pred = model.predict(train_x)
test_pred = model.predict(test_x)

In [ ]:
plot(train_x, train_y)
plot(test_x, test_y)
plot(train_x, train_pred, 'r')
plot(test_x, test_pred, 'r')

# Generate Example Data

#### LSTM models are best when there is a lot of autocorrelation

In [ ]:

t = np.linspace(0, 200, num=1000)
x = np.random.randn(len(t)+1)
y = 2.0*np.sin(t) + 1.0*np.random.randn(len(t))# + np.roll(x, 1)[1:]

train_t = t[0:700]
train_y = y[0:700]
val_t = t[700:900]
val_y = y[700:900]
test_t = t[900:]
test_y = y[900:]

plot(t, y)

plot(train_t, train_y)
_ = plot(test_t, test_y)


In [ ]:
_ = pd.plotting.autocorrelation_plot(y)

# Convert data into sequences

#### LSTM models require a sequence of previous timesteps to perform a forecast, so we need to transform the data. Luckily Keras has a built in function for doing this

In [ ]:
seq_len = 10

train_data = TimeseriesGenerator(train_y[:, np.newaxis], train_y,
                               length=seq_len, sampling_rate=1,
                               batch_size=10)
val_data = TimeseriesGenerator(val_y[:, np.newaxis], val_y,
                               length=seq_len, sampling_rate=1,
                               batch_size=10)

test_data = TimeseriesGenerator(test_y[:, np.newaxis], test_y,
                               length=seq_len, sampling_rate=1,
                               batch_size=10)


In [ ]:
_x, _y = train_data[0]


In [ ]:
train_y[0:12]

In [ ]:
_x[0:2]

In [ ]:
_y[0:2]

# Build LSTM model

#### Keras is a nice high-level wrapper on other packages (such as tensorflow)

In [ ]:

model = Sequential()
model.add(LSTM(32, input_shape=[seq_len, 1]))
model.add(Dense(1))
model.compile(loss='mse', optimizer='adam')

history = model.fit_generator(train_data, epochs=8, validation_data=val_data, verbose=1)



#### If the model is converging towards a good fit, you will see the validation error decrease and start to flatten out

In [ ]:
plot(history.history['val_loss'])
xlabel('Epochs')
_ = ylabel('Validation Error')

# Predict future timesteps

#### To do multi-step prediction, we feed in the previous predictions as the input to the next prediction. Errors will start to build up as we get further from the known data

In [ ]:
preds = []
fdata = val_y[-seq_len:]
for i in range(test_t.shape[0]):
    pred = model.predict(fdata.reshape((1, seq_len, 1)))
    preds.append(pred[0][0])
    fdata = np.roll(fdata, -1)
    fdata[-1] = pred

plot(test_t, test_y)
plot(test_t, preds)

_ = legend(['actual', 'prediction'])